In [ ]:
!git clone https://github.com/pytorch/fairseq -q
%cd fairseq
!pip uninstall numpy -q -y
!pip install wandb -q
!pip install --editable ./ -q
%cd ..

/content/fairseq
     |████████████████████████████████| 1.8MB 7.7MB/s 
     |████████████████████████████████| 174kB 37.1MB/s 
     |████████████████████████████████| 133kB 34.5MB/s 
     |████████████████████████████████| 102kB 13.6MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 133kB 7.9MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 112kB 56.0MB/s 
     |████████████████████████████████| 645kB 47.5MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 whi

In [ ]:
import torch

torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/spm.128k.model"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/data_dict.128k.txt"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/model_dict.128k.txt"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/language_pairs_small_models.txt"
!wget "https://dl.fbaipublicfiles.com/m2m_100/418M_last_checkpoint.pt"

--2021-06-25 14:25:14--  https://dl.fbaipublicfiles.com/m2m_100/418M_last_checkpoint.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4839402958 (4.5G) [application/octet-stream]
Saving to: ‘418M_last_checkpoint.pt’

418M_last_checkpoin 100%[===================>]   4.51G  52.8MB/s    in 95s     

2021-06-25 14:26:49 (48.7 MB/s) - ‘418M_last_checkpoint.pt’ saved [4839402958/4839402958]



In [ ]:
! pip install sentencepiece -q

     |████████████████████████████████| 1.2MB 7.5MB/s 


In [ ]:
import pandas as pd
import torch
import numpy as np
import os
import random

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(7)

In [ ]:

PATH_TO_DATASET = "./"  #Where you stored the dataset

train = pd.read_csv(os.path.join(PATH_TO_DATASET, "Train.csv"))

#Remove any possible duplicates
train = train.drop_duplicates(subset=["Yoruba", "English"])

#Lowercase and remove trailing spaces
train["Yoruba"] = train.apply(lambda x: (x.Yoruba).strip().lower(), axis=1)
train["English"] = train.English.apply(lambda x: x.lower())

train = train[["Yoruba", "English"]]
train.columns = ["input_text", "target_text"]

#Train 95% / Validation 5% Split
validation = train.sample(frac=0.05).astype(str)
train = train.drop(index=validation.index).astype(str)

In [ ]:
train_txt = "\n".join(train.input_text.values.tolist())

file = open("yoruba_txt_train.txt", "w")
file.write(train_txt)
file.close()


train_target_txt = "\n".join(train.target_text.values.tolist())

file = open("english_txt_train.txt", "w")
file.write(train_target_txt)
file.close()

In [ ]:
validation_txt = "\n".join(validation.input_text.values.tolist())

file = open("yoruba_txt_validation.txt", "w")
file.write(validation_txt)
file.close()


validation_target_txt = "\n".join(validation.target_text.values.tolist())

file = open("english_txt_validation.txt", "w")
file.write(validation_target_txt)
file.close()

In [ ]:
!python fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=yoruba_txt_train.txt \
        --outputs=train.yo

skipped 0 empty lines
filtered 0 lines


In [ ]:
!python fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=english_txt_train.txt \
        --outputs=train.en

skipped 0 empty lines
filtered 0 lines


In [ ]:
!python fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=yoruba_txt_validation.txt \
        --outputs=val.yo

skipped 0 empty lines
filtered 0 lines


In [ ]:
!python fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=english_txt_validation.txt \
        --outputs=val.en

skipped 0 empty lines
filtered 0 lines


In [ ]:
!fairseq-preprocess \
    --source-lang yo --target-lang en \
    --trainpref train \
    --validpref val \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict model_dict.128k.txt --tgtdict model_dict.128k.txt

2021-06-25 14:26:58 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='yo', srcdict='model_dict.128k.txt', suppress_crashes=Fals

In [ ]:
!mkdir checkpoint

In [ ]:
import threading 
import os


def remove_checkpoints():
    threading.Timer(5, remove_checkpoints).start()
    files = os.listdir("checkpoint")
    #print("here")
  
    for file in files:
        if file != "checkpoint_best.pt" and file.split(".")[-1] =="pt":
            os.remove("checkpoint/"+file)
            print("Removed " + file)

remove_checkpoints()

In [ ]:
import wandb

wandb.login(key="PUT YOUR API KEY HERE")

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!fairseq-train data_bin \
  --finetune-from-model  "418M_last_checkpoint.pt"\
  --save-dir checkpoint \
  --task translation_multi_simple_epoch \
  --encoder-normalize-before \
  --lang-pairs 'yo-en' \
  --batch-size 10 \
  --decoder-normalize-before \
  --encoder-langtok src \
  --decoder-langtok \
  --criterion cross_entropy \
  --optimizer adafactor \
  --lr-scheduler cosine \
  --lr 3e-05 \
  --max-update 40000 \
  --update-freq 2 \
  --save-interval 1 \
  --save-interval-updates 5000 \
  --keep-interval-updates 10 \
  --no-epoch-checkpoints \
  --log-format simple \
  --log-interval 2 \
  --patience 10 \
  --arch transformer_wmt_en_de_big \
  --encoder-layers 12 --decoder-layers 12 \
  --share-decoder-input-output-embed --share-all-embeddings \
  --ddp-backend no_c10d \
  --max-epoch 10 \
  --wandb-project "Yoruba M2M"

2021-06-25 14:27:04 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 2, 'log_format': 'simple', 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'Yoruba M2M', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_proce

In [ ]:
!rm -rf data_bin

In [ ]:
!fairseq-preprocess \
    --source-lang yo --target-lang en \
    --testpref val \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt

2021-06-25 14:44:49 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='yo', srcdict='data_dict.128k.txt', suppress_crashes=False

In [ ]:
!fairseq-generate "data_bin/"  --batch-size 32 \
      --path "checkpoint/checkpoint_best.pt" \
      --fixed-dictionary model_dict.128k.txt \
      -s yo -t en \
      --remove-bpe 'sentencepiece' \
      --beam 5 \
      --task translation_multi_simple_epoch \
      --lang-pairs language_pairs_small_models.txt \
      --decoder-langtok \
      --encoder-langtok src \
      --gen-subset test > gen_out

  0% 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
